<a href="https://colab.research.google.com/github/Arch0290/BeZenAssignment/blob/main/BeZen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Be Zen Assignment

This assignment attempts to solve the queries given in the assignment.

The first section deals with exploring the data

In [42]:
#importing the libraries

import pandas as pd
import numpy as np
import re

In [3]:
#Reading the csv file

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/2022_02_08-02_30_31_AM.csv")
df.head()

,uuid,price_string,price_string_unf,product_type,level_1,category
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN,NaN,TGFwdG9wIENvdmVycy9CYWdz,TGFwdG9wIENhc2U,RWxlY3Ryb25pY3M
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN,NaN,QmFraW5nIEN1cHMgYW5kIE1hdHM,QmFraW5nIE1hdHMgLyBCYWtpbmcgZGlzaA,a2l0Y2hpbmcgYW5kIGRpbmluZw
2,acbd66ff-79f8-467a-91f9-108a45af5626,$19.95,NaN,R3VtbWllcyB2aXRhbWlucyBhbmQgbWluZXJhbHMgZm9yIG...,SW1tdW5pdHkgZ3VtbWllcw,SGVhbHRo
3,963915d6-b2e3-4098-b242-9410593cf205,$92.00,NaN,U2VydW1z,RmFjZSBTZXJ1bQ,YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl
4,b5b68f3c-b1e0-40e5-8ee5-e2f7236c1daf,11.50,NaN,RWF0aW5nIFV0ZW5zaWxzL0N1dGxlcnk,Q2hvcHN0aWNrcw,a2l0Y2hpbmcgYW5kIGRpbmluZw


In [5]:
df.info()
print("\nExploring number of null elements in the dataset\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474697 entries, 0 to 474696
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   uuid              474697 non-null  object
 1   price_string      190611 non-null  object
 2   price_string_unf  10747 non-null   object
 3   product_type      474345 non-null  object
 4   level_1           472988 non-null  object
 5   category          474345 non-null  object
dtypes: object(6)
memory usage: 21.7+ MB

Exploring number of null elements in the dataset
 uuid                     0
price_string        284086
price_string_unf    463950
product_type           352
level_1               1709
category               352
dtype: int64


#Observation

- The dataset contains 6 columns describing a product
- It contains null values that needs to be explored

Q1. To find the Products without prices

- The price_string column has 284086 null elements.

In [31]:
#Getting the total null elements for 'price_string' column
print("Total products without prices = ", df['price_string'].isnull().sum())

Total products without prices =  284086


In [6]:
#Creating a copy of the dataframe to view the entries with null 'price_string' values
df1 = df.copy(deep=True)
NaN_prices_product = df1[df1['price_string'].isnull()]
df2 = NaN_prices_product.iloc[:, 0:2]

In [7]:
print(df2.info())

df2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284086 entries, 0 to 474696
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   uuid          284086 non-null  object
 1   price_string  0 non-null       object
dtypes: object(2)
memory usage: 6.5+ MB
None


,uuid,price_string
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN
5,389d9f75-cc3f-4bd2-94f7-93e381a3bed5,NaN
13,1f2766ec-0a27-43f8-b28b-77a44492ead1,NaN
16,7bad39f5-74b9-4611-afb9-46fb135190cd,NaN


Q2. Count of products without prices and with prices in each Product Type, Category, Level 1

In [36]:
df.groupby('category')['price_string'].value_counts()

category                          price_string
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz  $89             3893
                                  $160            2476
                                  $199            2086
                                  $79             2080
                                  $75.00          1959
                                                  ... 
cGV0IHN1cHBsaWVz                  $10.00           361
                                  $16.99           343
                                  $30.00           339
                                  $28.00           337
                                  $49.00           324
Name: price_string, Length: 336, dtype: int64

In [40]:
df.groupby('category')['price_string'].value_counts().unstack().fillna(0)

price_string,$0.00,$10.00,$10.29,$10.99,$100,$100.00,$1099,$11.00,$11.30,$11.75,...,14.95,17.00,21.00,22.00,32.99,5.75,52.00,6.60,67.00,7.00
category,,,,,,,,,,,,,,,,,,,,,
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz,0.0,0.0,0.0,0.0,342.0,693.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,334.0,0.0,0.0,0.0
QXJ0IHN1cHBsaWVz,0.0,331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
QmFieWNhcmU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RWxlY3Ryb25pY3M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SG91c2Vob2xkIGFuZCBDbGVhbmluZw,3175.0,0.0,0.0,335.0,0.0,0.0,348.0,366.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SGVhbHRo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VG95cyBhbmQgR2FtZXM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1418.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl,4196.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Q3. Correct Product Prices in the correct format (eg: $56) wherever possible and separate them into currency and value columns.

In [41]:
#understanding the pattern of 'price_string'
df['price_string'].unique()

array([nan, '$19.95', '$92.00', '11.50', '$24.99', '$148.00', '$89',
       '$14.95', '$55.00', '32.99', '52.00', '$11.30', '$13.99', '$6.95',
       '$0.00', '$129', '$10.99', '$6.49', '$18.99', '$179', '$9.99',
       '$3.99', '$549', '$16.99', '$49.99', '$90.00', '$12.99', '$59',
       '$85.00', '$128.00', '$150', '$6.99', '$34.00', '$248.00', '$4.99',
       '$65', '11.00', '$459', '$199', '14.95', '$135.05', '$1099',
       '$7.97', '$6.75', '$15.00', '$5.29', '$10.00', '7.00', '$80.00',
       '$2.29', '$119', '6.60', '$399', '$39.95', '12.00', '$8.99', '$79',
       '$35', '$34.95', '$99', '$18.95', '$459.99', '$58.00', '$25.70',
       '$3.50', '$619.00', '$95', '$159', '$45', '$149', '$85', '$8.95',
       '$69', '$14.00', '$38.00', '$6.50', '$12.50', '$24.00', '$27.00',
       '$82.75', '$9.95', '$26.96', '$17.96', '$15.99', '$6.17', '$75',
       '$250', '$29.00', '$1198', '$18.00', '$45.00', '$44.00', '$14.99',
       '$249', '$86.00', '$160', '$14.37', '$7.99', '$178', '$

The string is in the format \$ [float number]. We use regular expression the extract the required values, i.e the numbers (final_price) and ($) currencies.

In [40]:
#initialising and extracting the required patterns to form new columns

pattern1 = r'([0-9]+.[0-9]+)'
pattern2 = r'([$])'

df['currency'] = df['price_string'].str.extract(pattern2) 
df['final_price'] = df['price_string'].str.extract(pattern1)
len(df['final_price'].unique())

220

In [48]:
#Display the 'uuid' with prices and currencies
df3 = df.loc[:,['uuid','currency','final_price']]
df3.head()

,uuid,currency,final_price
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN,NaN
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN,NaN
2,acbd66ff-79f8-467a-91f9-108a45af5626,$,19.95
3,963915d6-b2e3-4098-b242-9410593cf205,$,92.00
4,b5b68f3c-b1e0-40e5-8ee5-e2f7236c1daf,NaN,11.50


In [54]:
df3.groupby('uuid')['final_price']

In [39]:
len(df['category'].unique())

14

In [56]:
df['final_price'] = df['final_price'].astype('float')

In [57]:
avg_price = df.groupby('category')['final_price'].agg(np.mean)

In [58]:
avg_price

category
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz       157.243994
QXJ0IHN1cHBsaWVz                        19.042907
QmFieWNhcmU                            160.559798
RWxlY3Ryb25pY3M                         63.468416
SG91c2Vob2xkIGFuZCBDbGVhbmluZw          77.629610
SGVhbHRo                                32.512631
VG95cyBhbmQgR2FtZXM                     32.100077
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ     84.800852
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl        34.949780
Z3JvY2VyaWVz                            10.482517
a2l0Y2hpbmcgYW5kIGRpbmluZw              16.580869
b2ZmaWNlIHByb2R1Y3Rz                    20.450415
cGV0IHN1cHBsaWVz                        13.976608
Name: final_price, dtype: float64